<a href="https://colab.research.google.com/github/IanGZC/Diplomado-/blob/main/Presentaci%C3%B3n_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La presentación se puede ver en: https://www.canva.com/design/DAGANt9W7MY/pPLahiJ6A2tDoCJgtoKeeQ/edit?utm_content=DAGANt9W7MY&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from scipy import stats

In [ ]:
d=pd.read_csv('/content/Crystal_structure.csv')
df=d.drop(columns=['Compound', 'A', 'B', 'v(A)', 'v(B)','In literature','τ'])
tar=d['Lowest distortion']
pred=df.loc[df['Lowest distortion']=='-']
df=df.drop(df.loc[df['Lowest distortion']=='-'].index)
est=pd.DataFrame(d['In literature'].value_counts()).reset_index()
est
fig=px.bar(x=est['In literature'], y=est['count'], labels={'x': 'Artículos', 'y': 'Número de perovskitas'}, title='Perovskitas reportadas en la literatura')
fig.show()

In [ ]:
d.A.nunique()

73

In [ ]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5329 entries, 0 to 5328
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Compound           5329 non-null   object 
 1   A                  5329 non-null   object 
 2   B                  5329 non-null   object 
 3   In literature      5329 non-null   bool   
 4   v(A)               5329 non-null   object 
 5   v(B)               5329 non-null   object 
 6   r(AXII)(Å)         5329 non-null   float64
 7   r(AVI)(Å)          5329 non-null   float64
 8   r(BVI)(Å)          5329 non-null   float64
 9   EN(A)              5329 non-null   float64
 10  EN(B)              5329 non-null   float64
 11  l(A-O)(Å)          5329 non-null   float64
 12  l(B-O)(Å)          5329 non-null   float64
 13  ΔENR               5329 non-null   float64
 14  tG                 5329 non-null   float64
 15  τ                  5329 non-null   object 
 16  μ                  5329 

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 5276 entries, 0 to 5328
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   r(AXII)(Å)         5276 non-null   float64
 1   r(AVI)(Å)          5276 non-null   float64
 2   r(BVI)(Å)          5276 non-null   float64
 3   EN(A)              5276 non-null   float64
 4   EN(B)              5276 non-null   float64
 5   l(A-O)(Å)          5276 non-null   float64
 6   l(B-O)(Å)          5276 non-null   float64
 7   ΔENR               5276 non-null   float64
 8   tG                 5276 non-null   float64
 9   μ                  5276 non-null   float64
 10  Lowest distortion  5276 non-null   object 
dtypes: float64(10), object(1)
memory usage: 494.6+ KB


,r(AXII)(Å),r(AVI)(Å),r(BVI)(Å),EN(A),EN(B),l(A-O)(Å),l(B-O)(Å),ΔENR,tG,μ
count,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000,5276.000000
mean,0.975038,0.825679,0.815317,1.573211,1.575324,2.290079,2.011475,-2.194221,0.766848,0.582370
std,0.337024,0.244605,0.243546,0.449922,0.448241,0.663045,0.594176,0.661116,0.135688,0.173961
min,0.270000,0.270000,0.270000,0.790000,0.790000,0.000000,0.000000,-5.411536,0.384648,0.192857
25%,0.690000,0.630000,0.630000,1.200000,1.200000,2.290644,1.956808,-2.599286,0.669561,0.450000
50%,0.930000,0.800000,0.770000,1.550000,1.550000,2.393444,2.096141,-2.098589,0.757121,0.550000
75%,1.180000,0.950000,0.950000,1.930000,1.930000,2.571812,2.286435,-1.710080,0.851209,0.678571
max,1.880000,1.670000,1.670000,2.540000,2.540000,3.300176,3.009747,-0.601714,1.388809,1.192857


In [ ]:
class_counts = df['Lowest distortion'].value_counts()
class_names = sorted(df['Lowest distortion'])

print("Class Distribution:")
for i, count in enumerate(class_counts):
    class_name = class_names[i]
    percentage = (count / len(df)) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

fig = px.bar(x=class_counts.index, y=class_counts.values, labels={'x': 'Lowest distortion', 'y': 'Count'}, title='Target Class Distribution', template = 'plotly_white', text = class_counts.values)
fig.show()

Class Distribution:
cubic: 3253 (61.66%)
cubic: 1573 (29.81%)
cubic: 323 (6.12%)
cubic: 127 (2.41%)


In [ ]:
corr_matrix = df.drop(columns=['Lowest distortion']).corr()

fig = px.imshow(corr_matrix, color_continuous_scale = 'RdBu', range_color=[-1,1], text_auto = True, aspect = "auto", labels=dict(color="Correlation"), title='Feature Correlation', height = 800)
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
px.scatter(df, x=df['r(BVI)(Å)'], y=df['μ'], color='Lowest distortion')

In [ ]:
fig = px.scatter_matrix(df, dimensions=df.columns[:-1], color='Lowest distortion', template = 'plotly_white')
fig.update_traces(diagonal_visible=False)
fig.update(layout_coloraxis_showscale=False)
fig.show()

Predicción

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['Lowest distortion'], test_size=0.2, random_state=7)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Regresión logística

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [ ]:
lr_pred = lr.predict(X_test_scaled)

lr_report = classification_report(y_test, lr_pred)
print("Logistic Regression Classification Report:")
print(lr_report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Logistic Regression Classification Report:
              precision    recall  f1-score   support

       cubic       0.66      0.91      0.77       648
orthorhombic       0.53      0.27      0.36       318
rhombohedral       0.00      0.00      0.00        67
  tetragonal       0.00      0.00      0.00        23

    accuracy                           0.64      1056
   macro avg       0.30      0.30      0.28      1056
weighted avg       0.57      0.64      0.58      1056



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
lr_accuracy = accuracy_score(y_test, lr_pred)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")

lr_precision = precision_score(y_test, lr_pred, average='weighted')
print(f"Logistic Regression Precission: {lr_precision:.2f}")

lr_recall = recall_score(y_test, lr_pred, average='weighted')
print(f"Logistic Regression Recall: {lr_recall:.2f}")

lr_f1 = f1_score(y_test, lr_pred, average='weighted')
print(f"Logistic Regression F1 Score: {lr_f1:.2f}")

Logistic Regression Accuracy: 0.64
Logistic Regression Precission: 0.57
Logistic Regression Recall: 0.64
Logistic Regression F1 Score: 0.58


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                       x=labels, y=labels, color_continuous_scale='Viridis', text_auto = True,
                       title="Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale=False)
    fig_cm.show()

In [ ]:
lr_cm = confusion_matrix(y_test, lr_pred)
plot_confusion_matrix(lr_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

K-nn

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [ ]:
knn_pred = knn.predict(X_test_scaled)
knn_report = classification_report(y_test, knn_pred)
print("KNN Classification Report:")
print(knn_report)

KNN Classification Report:
              precision    recall  f1-score   support

       cubic       0.77      0.88      0.82       648
orthorhombic       0.66      0.55      0.60       318
rhombohedral       0.38      0.21      0.27        67
  tetragonal       0.31      0.17      0.22        23

    accuracy                           0.72      1056
   macro avg       0.53      0.45      0.48      1056
weighted avg       0.70      0.72      0.71      1056



In [ ]:
knn_cm = confusion_matrix(y_test, knn_pred)
plot_confusion_matrix(knn_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

In [ ]:
knn_accuracy = accuracy_score(y_test, knn_pred)
print(f"KNN Accuracy: {knn_accuracy:.2f}")

knn_precision = precision_score(y_test, knn_pred, average='weighted')
print(f"KNN Precission: {knn_precision:.2f}")



knn_recall = recall_score(y_test, knn_pred, average='weighted')
print(f"KNN Recall: {knn_recall:.2f}")



knn_f1 = f1_score(y_test, knn_pred, average='weighted')
print(f"KNN F1 Score: {knn_f1:.2f}")

KNN Accuracy: 0.72
KNN Precission: 0.70
KNN Recall: 0.72
KNN F1 Score: 0.71


SVV

In [ ]:
svm = SVC()
svm.fit(X_train_scaled, y_train)

SVC()

In [ ]:
svm_pred = svm.predict(X_test_scaled)
svm_report = classification_report(y_test, svm_pred)
print("SVM Classification Report:")
print(svm_report)

SVM Classification Report:
              precision    recall  f1-score   support

       cubic       0.73      0.89      0.80       648
orthorhombic       0.60      0.49      0.54       318
rhombohedral       0.00      0.00      0.00        67
  tetragonal       0.00      0.00      0.00        23

    accuracy                           0.70      1056
   macro avg       0.33      0.35      0.34      1056
weighted avg       0.63      0.70      0.66      1056



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
svm_cm = confusion_matrix(y_test, svm_pred)
plot_confusion_matrix(svm_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

Naibe-Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train_scaled, y_train)

GaussianNB()

In [ ]:
nb_pred = nb.predict(X_test_scaled)
nb_report = classification_report(y_test, nb_pred)
print("Naive Bayes Classification Report:")
print(nb_report)

Naive Bayes Classification Report:
              precision    recall  f1-score   support

       cubic       0.68      0.70      0.69       648
orthorhombic       0.46      0.40      0.43       318
rhombohedral       0.20      0.16      0.18        67
  tetragonal       0.14      0.39      0.21        23

    accuracy                           0.57      1056
   macro avg       0.37      0.41      0.38      1056
weighted avg       0.57      0.57      0.57      1056



In [ ]:
nb_cm = confusion_matrix(y_test, nb_pred)
plot_confusion_matrix(nb_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

DT

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_scaled, y_train)

DecisionTreeClassifier()

In [ ]:
dt_pred = dt.predict(X_test_scaled)
dt_report = classification_report(y_test, dt_pred)
print("Decision Trees Classification Report:")
print(dt_report)

Decision Trees Classification Report:
              precision    recall  f1-score   support

       cubic       0.81      0.82      0.82       648
orthorhombic       0.64      0.58      0.61       318
rhombohedral       0.24      0.30      0.27        67
  tetragonal       0.20      0.26      0.23        23

    accuracy                           0.70      1056
   macro avg       0.47      0.49      0.48      1056
weighted avg       0.71      0.70      0.71      1056



In [ ]:
dt_cm = confusion_matrix(y_test, dt_pred)
plot_confusion_matrix(dt_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

rf

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [ ]:
rf_pred = rf.predict(X_test_scaled)
rf_report = classification_report(y_test, rf_pred)
print("Random Forest Classification Report:")
print(rf_report)

Random Forest Classification Report:
              precision    recall  f1-score   support

       cubic       0.83      0.88      0.85       648
orthorhombic       0.71      0.66      0.68       318
rhombohedral       0.48      0.31      0.38        67
  tetragonal       0.37      0.30      0.33        23

    accuracy                           0.77      1056
   macro avg       0.59      0.54      0.56      1056
weighted avg       0.76      0.77      0.76      1056



In [ ]:
rf_cm = confusion_matrix(y_test, rf_pred)
plot_confusion_matrix(rf_cm, ['cubic', 'orthorhombic', 'rhombohedral','tetragonal'])

PCA

In [ ]:
X = df.drop('Lowest distortion', axis=1)
pca = PCA(n_components=3)
X_pca2 = pca.fit_transform(X)
X_pca2

array([[ 0.1695476 ,  1.56877769,  2.9188454 ],
       [-0.09937042,  1.93871524,  0.59357783],
       [-0.79645694,  1.60527971,  1.00384841],
       ...,
       [ 0.6292399 ,  0.20318061, -0.05160799],
       [-0.09619509, -0.11874542, -0.09856008],
       [ 0.04047199, -0.05422841, -0.01347266]])

In [ ]:
components = pca.components_
components

array([[ 2.17441884e-01,  1.51482489e-01,  2.12275848e-01,
        -2.54187107e-01, -3.40297235e-01,  2.64556121e-01,
         2.32231116e-01, -7.46582926e-01,  3.33033120e-04,
         1.51625672e-01],
       [-2.07159887e-01, -1.27669216e-01,  1.44582399e-01,
         1.46833474e-01, -2.31388524e-01, -8.39878665e-01,
         2.91333729e-01, -1.75725063e-01, -1.13862264e-01,
         1.03272151e-01],
       [ 1.42865158e-01,  1.12643112e-01, -8.38890592e-03,
        -2.72239529e-01, -8.68064931e-02, -3.40389327e-01,
        -8.54292581e-01, -1.93215207e-01,  4.61762828e-02,
        -5.99177580e-03]])

In [ ]:
explained_variance = pca.explained_variance_
explained_variance

array([0.71871335, 0.46093884, 0.33248043])

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio

array([0.37540434, 0.24076141, 0.17366394])

In [ ]:
pca2 = df.copy()
pca2['PCA1'] = X_pca2[:, 0]
pca2['PCA2'] = X_pca2[:, 1]
pca2['PCA3'] = X_pca2[:, 2]

fig = px.scatter_3d(pca2, x='PCA1', y='PCA2', z='PCA3', color='Lowest distortion', title='PCA of Iris Dataset', template = 'plotly_white')
fig.update_layout(xaxis_title='Principal Component 1', yaxis_title='Principal Component 2')

fig.show()

Predicción

In [ ]:
predd=pred.drop(columns=['Lowest distortion'])

X_test_scal = scaler.transform(predd)

rf_predict = rf.predict(X_test_scal)
# rf_report = classification_report(y_test, rf_predict)
# print("Random Forest Classification Report:")
# print(rf_report)
rf_predict

predd['Estructura Cristalina']=0

for i in range(len(rf_predict)):
  predd['Estructura Cristalina'].iloc[i]=rf_predict[i]

predd

class_counts = predd['Estructura Cristalina'].value_counts()
class_names = sorted(predd['Estructura Cristalina'])

print("Class Distribution:")
for i, count in enumerate(class_counts):
    class_name = class_names[i]
    percentage = (count / len(predd)) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

fig = px.bar(x=class_counts.index, y=class_counts.values, labels={'x': 'Lowest distortion', 'y': 'Count'}, title='Target Class Distribution', template = 'plotly_white', text = class_counts.values)
fig.show()

<ipython-input-39-af982dde6ce0>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Class Distribution:
cubic: 28 (52.83%)
cubic: 17 (32.08%)
cubic: 6 (11.32%)
cubic: 2 (3.77%)


In [ ]:
svm_accuracy = accuracy_score(y_test, svm_pred)
print(f"SVM Accuracy: {svm_accuracy:.2f}")

svm_precision = precision_score(y_test, svm_pred, average='weighted')
print(f"SVM Precission: {svm_precision:.2f}")



svm_recall = recall_score(y_test, svm_pred, average='weighted')
print(f"SVM Recall: {svm_recall:.2f}")



svm_f1 = f1_score(y_test, svm_pred, average='weighted')
print(f"SVM F1 Score: {svm_f1:.2f}")

SVM Accuracy: 0.70
SVM Precission: 0.63
SVM Recall: 0.70
SVM F1 Score: 0.66


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
nb_accuracy = accuracy_score(y_test, nb_pred)
print(f"Naive Bayes Accuracy: {nb_accuracy:.2f}")

nb_precision = precision_score(y_test, nb_pred, average='weighted')
print(f"Naive Bayes Precission: {nb_precision:.2f}")



nb_recall = recall_score(y_test, nb_pred, average='weighted')
print(f"Naive Bayes Recall: {nb_recall:.2f}")



nb_f1 = f1_score(y_test, nb_pred, average='weighted')
print(f"Naive Bayes F1 Score: {nb_f1:.2f}")

Naive Bayes Accuracy: 0.57
Naive Bayes Precission: 0.57
Naive Bayes Recall: 0.57
Naive Bayes F1 Score: 0.57


In [ ]:
dt_accuracy = accuracy_score(y_test, dt_pred)
print(f"Decision Trees Accuracy: {dt_accuracy:.2f}")


dt_precision = precision_score(y_test, dt_pred, average='weighted')
print(f"Decision Trees Precission: {dt_precision:.2f}")


dt_recall = recall_score(y_test, dt_pred, average='weighted')
print(f"Decision Trees Recall: {dt_recall:.2f}")


dt_f1 = f1_score(y_test, dt_pred, average='weighted')
print(f"Decision Trees F1 Score: {dt_f1:.2f}")


Decision Trees Accuracy: 0.70
Decision Trees Precission: 0.71
Decision Trees Recall: 0.70
Decision Trees F1 Score: 0.71


In [ ]:
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")


rf_precision = precision_score(y_test, rf_pred, average='weighted')
print(f"Random Forest Precission: {rf_precision:.2f}")


rf_recall = recall_score(y_test, rf_pred, average='weighted')
print(f"Random Forest Recall: {rf_recall:.2f}")


rf_f1 = f1_score(y_test, rf_pred, average='weighted')
print(f"Random Forest F1 Score: {rf_f1:.2f}")

Random Forest Accuracy: 0.77
Random Forest Precission: 0.76
Random Forest Recall: 0.77
Random Forest F1 Score: 0.76


In [ ]:

models_data = {
    'Model': ['Logistic Regression', 'K-Nearest Neighbors', 'SVM', 'Naive Bayes', 'Decision Trees', 'Random Forest'],
    'Accuracy': [lr_accuracy, knn_accuracy, svm_accuracy, nb_accuracy, dt_accuracy, rf_accuracy],
    'Precision': [lr_precision, knn_precision, svm_precision, nb_precision, dt_precision, rf_precision],
    'Recall': [lr_recall, knn_recall, svm_recall, nb_recall, dt_recall, rf_recall],
    'F1-Score': [lr_f1, knn_f1, svm_f1, nb_f1, dt_f1, rf_f1]
}

df_models = pd.DataFrame(models_data)
df_models

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.640152,0.565799,0.640152,0.579098
1,K-Nearest Neighbors,0.724432,0.703452,0.724432,0.708308
2,SVM,0.696970,0.627805,0.696970,0.655872
3,Naive Bayes,0.565341,0.571703,0.565341,0.566248
4,Decision Trees,0.703598,0.711101,0.703598,0.706688
5,Random Forest,0.768939,0.757461,0.768939,0.761245


In [ ]:
fig_accuracy = px.bar(df_models, x='Model', y='Accuracy', title='Model Comparison - Accuracy', template = 'plotly_white', text = 'Accuracy')
fig_accuracy.update_layout(yaxis=dict(range=[0, 1]))
fig_accuracy.show()

In [ ]:
fig_precision = px.bar(df_models, x='Model', y='Precision', title='Model Comparison - Precision', template = 'plotly_white', text = 'Precision')
fig_precision.update_layout(yaxis=dict(range=[0, 1]))
fig_precision.show()

In [ ]:
fig_recall = px.bar(df_models, x='Model', y='Recall', title='Model Comparison - Recall', template = 'plotly_white', text = 'Recall')
fig_recall.update_layout(yaxis=dict(range=[0, 1]))
fig_recall.show()

In [ ]:
fig_f1_score = px.bar(df_models, x='Model', y='F1-Score', title='Model Comparison - F1-Score', template = 'plotly_white', text = 'Recall')
fig_f1_score.update_layout(yaxis=dict(range=[0, 1]))
fig_f1_score.show()

In [ ]:
dff=pd.read_excel('/content/Libro1.xlsx')
# dff=pd.DataFrame(dff.groupby(['Model','Test']).Score.max())
dff
fig_f1_score = px.bar(dff, x='Model', y='Score', title='Model Comparison', color='Test',template = 'plotly_white', barmode="group")
fig_f1_score.update_layout(yaxis=dict(range=[0, 1]))
fig_f1_score.show()